# LIBRARY

In [9]:
import dill  # in order to save Lambda Layer
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from resnet import *

# Define the model

In [10]:
model = resnet20()
# remember to set map_location
check_point = torch.load('D:/Documents/Kuliah/S1/GARUDA ACE/Coding/2-24-23_TMR/pretrained_models/resnet20-12fca82f.th', map_location={'cuda:1': 'cuda:0'})

model = torch.nn.DataParallel(model)
model.load_state_dict(check_point['state_dict'])

torch.save(model.module, 'D:/Documents/Kuliah/S1/GARUDA ACE/Coding/2-24-23_TMR/save_temp/resnet20_check_point.pth', pickle_module=dill)

# load the converted pretrained model
resnet = torch.load('D:/Documents/Kuliah/S1/GARUDA ACE/Coding/2-24-23_TMR/save_temp/resnet20_check_point.pth', map_location={'cuda:1': 'cuda:0'})
#resnet = torch.hub.load('pytorch/vision:v0.9.0', 'resnet20', pretrained=True)

# Load the CIFAR-10 dataset

In [11]:
transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False, num_workers=2)

Files already downloaded and verified


# Define the TMR class that implements TMR for ResNet-20


In [12]:
class TMR_ResNet(nn.Module):
    def __init__(self, resnet):
        super(TMR_ResNet, self).__init__()
        self.model1 = resnet
        self.model2 = resnet
        self.model3 = resnet
        
    def forward(self, x):
        out1 = self.model1(x)
        out2 = self.model2(x)
        out3 = self.model3(x)
        return out1, out2, out3

# Define the hook function to capture the output from each layer

In [13]:
# Create an instance of TMR_ResNet
tmr_resnet = TMR_ResNet(resnet)
tmr_resnet = tmr_resnet.cuda()

layer_outputs = {}
def get_layer_output(name):
    def hook(model, input, output):
        layer_outputs[name] = output[0].cpu().detach().numpy()
    return hook

# Register hooks for all layers in the model
layer_outputs = {}
for name, layer in tmr_resnet.named_modules():
    layer.register_forward_hook(get_layer_output(name))

tmr_resnet.eval()
criterion = nn.CrossEntropyLoss().cuda()
correct = 0
total = 0
# Get the outputs from each redundant model
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.cuda(), labels.cuda()
        # calculate outputs by running images through the network
        output1, output2, output3 = tmr_resnet(images)
        
        loss = criterion(output1, labels) + criterion(output2, labels) + criterion(output3, labels)
        _, predicted1 = torch.max(output1.data, 1)
        _, predicted2 = torch.max(output2.data, 1)
        _, predicted3 = torch.max(output3.data, 1)
        predicted = torch.mode(torch.stack((predicted1, predicted2, predicted3), dim=1), dim=1)[0]
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the test images:', (correct / total) * 100, '%')


Accuracy of the network on the test images: 83.85000000000001 %


# Print the outputs from each layer for each model

In [14]:
for i in range(1, 4):
    print(f"Output from Model {i}:")
    for name, output in layer_outputs.items():
        print(f"{name}: {output[i-1]}")

Output from Model 1:
model1.conv1: [[-0.7469853  -0.75809324 -0.66863525 ... -0.3237388  -0.218283
  -0.74341655]
 [ 0.09818165 -0.07078709 -0.01279279 ...  0.19948514  0.22555143
  -0.4208463 ]
 [ 0.13203412 -0.01480437 -0.08200161 ...  0.5296279   0.6376877
  -0.11754758]
 ...
 [ 0.9319954   0.9632476  -0.3325374  ... -0.28180268 -0.86972404
   0.13232331]
 [ 0.35840037  1.7566726   0.70661736 ...  0.3327812   0.45869046
  -0.54068446]
 [-0.2933924  -0.23364334  0.67456305 ... -0.23449083  0.11473389
   0.9249182 ]]
model1.bn1: [[0.84802556 0.8434594  0.8802332  ... 1.022011   1.0653611  0.8494926 ]
 [1.1954515  1.1259929  1.1498327  ... 1.2370946  1.2478098  0.98209274]
 [1.2093673  1.1490059  1.1213828  ... 1.3728076  1.4172281  1.1067709 ]
 ...
 [1.5382103  1.5510572  1.0183942  ... 1.0392499  0.7975709  1.2094861 ]
 [1.3024205  1.8772134  1.4455633  ... 1.2918892  1.3436471  0.93283045]
 [1.0344857  1.059047   1.4323866  ... 1.0586985  1.2022556  1.5353011 ]]
model1.layer1.0.conv